In [1]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os


from langdetect import detect_langs
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer



In [2]:
# #twitter api keys
# auth = tweepy.OAuth1UserHandler(
#    )

# api = tweepy.API(auth, wait_on_rate_limit=True)


In [3]:
# search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
# search_query = search_words + " -filter:retweets AND -filter:replies"


# limit = 100000
# tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(limit)

# data=[]
# for tweet in tweets:
#     data.append([tweet.id,tweet.user.screen_name, tweet.created_at, 
#                   tweet.full_text,tweet.user.location, tweet.retweet_count,tweet.favorite_count,
#                    tweet.user.followers_count, tweet.user.friends_count, tweet.user.verified, tweet.source])

In [4]:
# columns =['id','username','time_of_tweet', 'tweet', 'location', 
#          'retweets', 'likes', 'followers', 'following', 'verified', 'tweet_source']


# #convert to dataframe and import into csv
# bp_tweets=pd.DataFrame(data, columns=columns)
# bp_tweets.to_csv("BlackPanther-24-11.csv")

In [5]:
#save scrapped data to csv 
tweets = pd.read_csv('excel_outputs//BlackPanther-24-11.csv')
tweets.sample(10)

,Unnamed: 0,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source
2493,2493,1595343557774172161,RyanMECHX43,2022-11-23 09:08:38+00:00,"Just saw Black Panther: Wakanda Forever, and n...",NaN,0,0,2,12,False,Twitter for Android
16682,16682,1593974601519796230,dinuka_jeremy,2022-11-19 14:28:54+00:00,So watched #BlackPantherWakandaForever and was...,Sri Lanka,0,2,193,360,False,Twitter Web App
17463,17463,1593885939734319105,aoi_08H,2022-11-19 08:36:35+00:00,BLACKPanther終わったー！！！！\nめちゃくちゃ良かった😭\n#BlackPant...,NaN,0,0,1146,1139,False,Twitter for iPhone
27028,27028,1593094528764104704,jc_viceZero,2022-11-17 04:11:48+00:00,Another amazing track from Wakanda Forever!!\n...,NaN,0,1,219,960,False,Twitter Web App
29071,29071,1592991087420878849,AmalfiFranck,2022-11-16 21:20:46+00:00,C’est pas je regarde #BlackPanther2 qui c’est ...,"Athis-Mons, France",0,3,103,47,False,Twitter for iPhone
26863,26863,1593103924160245760,sterkinekor,2022-11-17 04:49:08+00:00,"""The Black Panther lives!!!!""🎞\n\nCatch the No...",South Africa,0,0,118552,479,True,Twitter Web App
18450,18450,1593794815967232001,imperfectstar28,2022-11-19 02:34:30+00:00,#WakandaForever is beautiful. A must see. The...,NaN,0,0,218,226,False,Twitter for iPhone
12337,12337,1594268500667019264,nose_troll,2022-11-20 09:56:45+00:00,When you're about to see #Wakandaforever &amp;...,NaN,0,0,62,106,False,Twitter for Android
29059,29059,1592991636090650624,the_exp1,2022-11-16 21:22:57+00:00,What can we really learn from the box office n...,NaN,0,0,279,842,False,Loomly
28478,28478,1593017653450661888,QBeast9922,2022-11-16 23:06:20+00:00,Bring love wherever you go… Shine light where...,NaN,0,1,906,2100,False,Twitter for iPhone


In [6]:
#column names and datatype
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35054 entries, 0 to 35053
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     35054 non-null  int64 
 1   id             35054 non-null  int64 
 2   username       35054 non-null  object
 3   time_of_tweet  35054 non-null  object
 4   tweet          35054 non-null  object
 5   location       24584 non-null  object
 6   retweets       35054 non-null  int64 
 7   likes          35054 non-null  int64 
 8   followers      35054 non-null  int64 
 9   following      35054 non-null  int64 
 10  verified       35054 non-null  bool  
 11  tweet_source   35054 non-null  object
dtypes: bool(1), int64(6), object(5)
memory usage: 3.0+ MB


In [7]:
#statistical description of the dataset
tweets.describe()

,Unnamed: 0,id,retweets,likes,followers,following
count,35054.000000,3.505400e+04,35054.000000,35054.000000,3.505400e+04,35054.000000
mean,17526.500000,1.593928e+18,4.522194,37.315513,3.897041e+04,1241.136247
std,10119.362505,8.477788e+14,63.704964,630.220989,5.791979e+05,6060.701871
min,0.000000,1.592672e+18,0.000000,0.000000,0.000000e+00,0.000000
25%,8763.250000,1.593155e+18,0.000000,0.000000,1.080000e+02,149.000000
50%,17526.500000,1.593876e+18,0.000000,1.000000,4.150000e+02,420.000000
75%,26289.750000,1.594521e+18,0.000000,3.000000,1.547000e+03,1047.000000
max,35053.000000,1.595755e+18,4870.000000,51334.000000,4.083612e+07,301618.000000


In [8]:
#drop unnamed column
tweets.drop(columns='Unnamed: 0', inplace = True)

#fill in missing location with 'None'
tweets['location'].fillna('None', inplace=True)

#change the datatype of tweet_time to datetime 
tweets['time_of_tweet'] = tweets['time_of_tweet'].astype('datetime64')

In [9]:
def get_hash(tweet):
    '''This function collect all the hashtags in the tweet column'''
    tweet = re.findall(r'\#\w+', tweet)
    return " ".join(tweet)

In [10]:
#applying the get_hash function
tweets['hashtags'] = tweets['tweet'].apply(get_hash)


In [11]:
#count the occurrence of each hashtags in the hashtags columns
hashtags = pd.Series(" ".join(tweets['hashtags']).split()).value_counts()
#hashtags.to_csv('excel_outputs//hashtag_freq.csv')    ##save the output as a csv
hashtags[:10]

#WakandaForever                20990
#BlackPanther                   9316
#BlackPantherWakandaForever     5887
#wakandaforever                 1997
#Namor                          1746
#blackpanther                   1297
#Marvel                         1211
#BlackPanther2                   998
#WakandaForeverID                978
#MarvelStudios                   808
dtype: int64

In [12]:
def get_languages(tweet):
    '''This function detects the language in each 
    tweet and returns it in ISO 639-1 codes'''
    try:
        tweet = detect_langs(tweet)
    except: 
        tweet = "nolang"
    tweet = str(tweet).split(':')[0][1:]
    return tweet

In [13]:
#applying the get_languages function
tweets['language'] = tweets['tweet'].apply(get_languages)

In [14]:
#substituting the short form language names for the long names
lang_code = pd.read_csv('language_codes.csv')
dict_lang_code = dict(zip(lang_code['alpha2'], lang_code['English']))
tweets['language'] = tweets['language'].replace(dict_lang_code)

In [15]:
#checking for changes
tweets.language.value_counts()[:10]

English       24191
Spanish        3348
Danish         1491
Portuguese     1451
French          773
Indonesian      527
Japanese        473
Thai            469
Norwegian       369
Italian         330
Name: language, dtype: int64

In [16]:
def get_links(tweets):
    '''This function iterate over the tweets and return 
    'Yes' is there's a link and 'No' if there's no link '''
    link = re.findall(r'((http|ftp|https):\/\/)(([\w.-]*)\.([\w]*))', tweets)
    if len(link) < 1:
        return 'No'
    else: return 'Yes'


In [17]:
#applying the get_links function
tweets['contains_media'] = tweets['tweet'].apply(get_links)

In [18]:
#checking for changes
tweets.contains_media.value_counts()

Yes    20872
No     14182
Name: contains_media, dtype: int64

In [19]:
cast = ['namor', 'shuri', 'ironheart', 'ramonda', 'namora', 
        'okoye', 'aneka', 'nakia', 'mbaku', 'attuma', 'everett', ]


def get_cast(tweets):
        '''This function extracts names of the cast into a new column'''      
        tweets = tweets.lower()
        tokens = word_tokenize(tweets)
        cast_names = [name for name in tokens if name in cast]
        cast_names_title = [name.title() for name in cast_names]
        return " ".join(cast_names_title)

In [20]:
#applying the get_cast function
tweets['cast_name'] = tweets['tweet'].apply(get_cast)

In [21]:
#count the occurrence of each name in the cast_name columns
names = pd.Series(" ".join(tweets['cast_name']).split()).value_counts()
# names.to_csv('excel_outputs//names_freq.csv') ##save the generated dataframe
names[:10]

Namor        3953
Shuri        1985
Okoye         348
Namora        295
Ironheart     288
Nakia         220
Ramonda       216
Mbaku         202
Attuma        123
Aneka          53
dtype: int64

In [22]:
#replace the '\n' characters with space
tweets['tweet'] = tweets['tweet'].replace('\n', ' ', regex=True)

In [23]:
#get the sentiment scores using Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()
tweets['sentiment_scores'] = tweets['tweet'].apply(sia.polarity_scores)
tweets['sentiment_scores'] = tweets['sentiment_scores'].apply(lambda x: x['compound'])

In [24]:
def get_sentiment(tweet):
    '''This functions returns Positive, Negative or 
    Neutral depending on the state of the sentiment score'''
    if tweet > 0:
        return "Positive"
    elif tweet == 0:
        return "Neutral"
    else: return "Negative"

In [25]:
#applying the get_sentiment function
tweets['sentiment'] = tweets['sentiment_scores'].apply(get_sentiment)

In [26]:
#checking for changes
tweets['sentiment'].value_counts()

Neutral     17445
Positive    12441
Negative     5168
Name: sentiment, dtype: int64

In [27]:
def get_country(inc_c, case, cor_c):
    '''This function substitutes the current location for the country name'''
    tweets.loc[tweets['c_location'].str.contains(inc_c, case=case), 'c_location'] = cor_c


In [28]:
#duplicate the location column to another column 
tweets['c_location'] = tweets['location']

#substitute incorrect spelling for correct spelling
dicts = {'brazil':'Brazil', 'brasil':'Brazil', 'nigeria':'Nigeria', 'méxico': 'Mexico', 'france':'France', 
        'india':'India', 'london':'United Kingdom', 'UK':'United Kingdom', 'england':'United Kingdom',
        'Bonaire':'Bonaire Islands', 'España':'Spain', 'thailand':'Thailand', 'Ohio':'United States',
        'South Africa':'South Africa'}
for key, val in dicts.items():
    get_country(key, False, val)

#substitute state names in the US
list_us = ['USA', 'KS', 'FL', 'CA', 'GA', 'AZ', 'CO','IL', 'TX', 'PA', 'DC', 'TN', 'NY', 'WA', 'NYC', 'NC', 'NV', 'OH', 'LA', 'MA', 'Los Angeles' 'new york','New York', 'Los Angeles']
for states in list_us:
    get_country(states, True, 'United States')

#minor formatting
tweets['c_location'].replace(['None', 'she/her'], np.nan, inplace=True)
tweets['c_location'].value_counts()[:10]

United States     5768
United Kingdom    1123
Brazil             666
Mexico             582
India              572
France             311
Spain              308
Nigeria            193
South Africa       190
Thailand           104
Name: c_location, dtype: int64

In [29]:
#frequency of tweet_source
tweets['tweet_source'].value_counts()[:10]

Twitter for iPhone      15298
Twitter for Android     11002
Twitter Web App          5560
TweetDeck                 575
Twitter for iPad          331
Instagram                 289
Hootsuite Inc.            218
Buffer                    174
IFTTT                     141
Twitter Media Studio      139
Name: tweet_source, dtype: int64

In [30]:
#frequency of verified or not
tweets['verified'].value_counts()

False    33570
True      1484
Name: verified, dtype: int64

In [31]:
#final sampling of the dataset
tweets.sample(20)

,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source,hashtags,language,contains_media,cast_name,sentiment_scores,sentiment,c_location
9319,1594480318882488324,bdirksmith,2022-11-20 23:58:26,"Wow, #WakandaForever was very, very good. I cr...",None,0,0,120,432,False,Twitter for iPhone,#WakandaForever,English,No,,0.6285,Positive,NaN
5461,1594893846797844481,ximmMunoz,2022-11-22 03:21:39,Por qué nadie ha hablado de lo hottttt que se ...,"Pachuca de Soto, Hidalgo",0,0,162,749,False,Twitter for Android,#WakandaForever,Portuguese,No,,0.3400,Positive,"Pachuca de Soto, Hidalgo"
134,1595714419157307392,BeeChance1,2022-11-24 09:42:19,The coonery of Black Panther. Did it empower t...,None,0,0,2,2,False,Twitter Web App,#BlackTwitter #thanksgivingwithblackfamilies #...,English,Yes,,0.0000,Neutral,NaN
23692,1593342555546128384,hellogem_,2022-11-17 20:37:22,Watching Black Panther again for the 100th tim...,None,0,1,142,539,False,Twitter for iPhone,#WakandaForever,English,No,,0.7717,Positive,NaN
5920,1594840616948666370,supermangeek101,2022-11-21 23:50:08,Disney will totally dominate Thanksgiving as B...,"Des Plaines, IL",2,3,2013,4973,False,Twitter for iPhone,#WakandaForever #StrangeWorld #BonesAndAll #Gl...,English,Yes,,0.4197,Positive,United States
35008,1592676403073294336,bms_te,2022-11-16 00:30:19,#Telugu #top5 movies by show count #Nov16Wed t...,None,0,1,918,95,False,crawlamma,#Telugu #top5 #Nov16Wed #Yashoda #KantaraTelug...,English,Yes,,0.0000,Neutral,NaN
919,1595579540956450817,perso_dev,2022-11-24 00:46:21,Me esperaba mas.. #WakandaForever,None,0,0,120,460,False,Twitter for Android,#WakandaForever,Portuguese,No,,0.0000,Neutral,NaN
3586,1595189146699862016,puregeekoficial,2022-11-22 22:55:04,#BlackPanther: #Wakanda Forever: Is There Some...,None,3,22,2220,204,False,dlvr.it,#BlackPanther #Wakanda,English,Yes,Shuri Namor,0.0000,Neutral,NaN
10716,1594399139994796032,adolino92,2022-11-20 18:35:52,Black Panther: Wakanda Forever mola #WakandaFo...,México,0,2,449,1253,False,Twitter Web App,#WakandaForever,English,Yes,,0.0000,Neutral,Mexico
32645,1592781609711042560,gabliotecaria,2022-11-16 07:28:22,Booked for #WakandaForever at 11.30am today. I...,None,0,1,675,608,False,Twitter for Android,#WakandaForever,English,Yes,,0.4215,Positive,NaN


In [32]:
#export to csv
tweets.to_csv('excel_outputs//BlackPanther_Cleaned.csv')